In [1]:
import numpy as np
from transformers import AutoTokenizer, AutoModel, DistilBertTokenizer, DistilBertForSequenceClassification
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import Dataset, TensorDataset, DataLoader
from transformers import get_scheduler
from eval_metrics import print_metrics_binary
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
# classifier = nn.Sequential(
#     nn.Linear(in_features=768, out_features=2, bias=True),
#     nn.Softmax(dim=1)
# )
# model = nn.Sequential(model, classifier)
model


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
criterion = torch.nn.BCELoss(weight=None, size_average=None, reduce=None, reduction='mean')
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

In [2]:
text = "you are a fool."
tokenized = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
# print(tokenized)
outputs = model(tokenized["input_ids"])
outputs

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.4749, -0.2102, -0.2954,  ...,  0.0859,  0.3189, -0.4628],
         [ 0.4604,  0.1088,  0.2628,  ...,  0.6170,  0.0967, -0.2721],
         [ 0.3512, -0.1699,  0.0030,  ...,  0.2668, -0.0499, -0.3658],
         ...,
         [-0.2661, -0.6352, -0.0016,  ...,  0.3084,  0.6158, -0.2966],
         [ 0.1039, -0.2937, -0.1192,  ...,  0.4055,  0.1102, -0.4169],
         [ 0.8741,  0.8321, -0.3052,  ..., -0.8622,  1.2682, -0.1773]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-0.0930, -0.1334,  0.9718, -1.0000,  0.9996,  0.9787, -0.0738, -0.6276,
         -0.0529,  0.1257,  0.9969,  0.9999, -0.9970, -0.7683, -0.1719, -0.5314,
          1.0000,  0.1479, -0.9952, -0.2338, -0.1088, -0.9090, -0.0530,  0.9998,
          0.1128,  0.0490,  0.9999,  0.9960, -0.3077, -0.0030, -0.1559, -0.9990,
          0.9995, -0.9990, -0.0295,  0.0740, -0.2439,  0.0292,  0.9845, -0.9993,
          0.0887, -0.9412,  0.29

In [4]:
print(outputs.last_hidden_state.shape)
print(outputs.pooler_output.shape)

torch.Size([1, 7, 768])
torch.Size([1, 768])


In [6]:
Classifier = nn.Sequential(
    nn.Linear(in_features=768, out_features=2, bias=True),
    nn.Softmax(dim=1)
)

pred = Classifier(outputs.pooler_output)
pred

tensor([[0.4887, 0.5113]], grad_fn=<SoftmaxBackward0>)